In [55]:
import os

from tokenizers import Tokenizer
from tokenizers.models import WordLevel, BPE
from tokenizers.trainers import BpeTrainer, WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace

from datasets import load_dataset

In [57]:
data_raw = load_dataset(
    path='opus_books', 
    name='it-ru',
    split='train'
)
data_raw['translation'][:10]

[{'it': 'Anna Karenina', 'ru': 'Анна Каренина'},
 {'it': 'Lev Tolstoj', 'ru': 'Толстой Лев Николаевич'},
 {'it': 'PARTE PRIMA', 'ru': 'ЧАСТЬ ПЕРВАЯ'},
 {'it': 'I', 'ru': 'I'},
 {'it': 'Tutte le famiglie felici sono simili le une alle altre; ogni famiglia infelice è infelice a modo suo.',
  'ru': 'Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.'},
 {'it': 'Tutto era sottosopra in casa Oblonskij.',
  'ru': 'Все смешалось в доме Облонских.'},
 {'it': 'La moglie era venuta a sapere che il marito aveva una relazione con la governante francese che era stata presso di loro, e aveva dichiarato al marito di non poter più vivere con lui nella stessa casa.',
  'ru': 'Жена узнала, что муж был в связи с бывшею в их доме француженкою-гувернанткой, и объявила мужу, что не может жить с ним в одном доме.'},
 {'it': 'Questa situazione durava già da tre giorni ed era sentita tormentosamente dagli stessi coniugi e da tutti i membri della famiglia e dai domestici

In [197]:
def _get_all_sentences(dataset, lang: str):
    """
    Iterator of sentences.
    """
    for item in dataset["translation"]:
        yield item[lang]


def _create_tokenizer(dataset, lang: str, algorithm: str = 'wordlevel') -> Tokenizer:
    """
    Creates tokenizer from text file and saves it.
    """
    unk_token = "[UNK]"
    special_tokens = ["[UNK]", "[PAD]", "[SOS]", "[EOS]"]

    if algorithm == 'bpe':
        model = BPE(unk_token=unk_token)
        trainer = BpeTrainer(special_tokens=special_tokens, min_frequency=5)
    else:
        model = WordLevel(unk_token=unk_token)
        trainer = WordLevelTrainer(special_tokens=special_tokens)
    
    tokenizer = Tokenizer(model=model)
    tokenizer.pre_tokenizer = Whitespace()
    tokenizer.train_from_iterator(iterator=_get_all_sentences(dataset, lang), trainer=trainer)

    return tokenizer


def make_tokenizer(dataset, lang: str, algorithm: str):
    """
    Create BPE (Byte-Pair Encoding) tokenizer.
    """
    # make sure that the tokenizer folder exists
    if not os.path.exists('tokenizers'):
        os.mkdir('tokenizers')

    # path to the tokenizer file
    tokenizer_path = os.path.join('tokenizers', f'tokenizer_{algorithm}_{lang}.json')
    
    if not os.path.exists(tokenizer_path):
        tokenizer = _create_tokenizer(dataset=dataset, lang=lang, algorithm=algorithm)
        tokenizer.save(tokenizer_path)
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))

    return tokenizer

In [198]:
src_tokenizer = make_tokenizer(dataset=data_raw, lang='ru', algorithm='bpe')
tgt_tokenizer = make_tokenizer(dataset=data_raw, lang='it', algorithm='bpe')

In [199]:
src_string = 'Все счастливые семьи похожи друг на друга, каждая несчастливая семья несчастлива по-своему.'
tgt_string = 'Tutte le famiglie felici sono simili le une alle altre; ogni famiglia infelice è infelice a modo suo.'

src_output = src_tokenizer.encode(src_string)
print(src_output.tokens)

tgt_output = src_tokenizer.encode(tgt_string)
print(tgt_output.tokens)

['Все', 'счастли', 'вые', 'семьи', 'похожи', 'друг', 'на', 'друга', ',', 'каждая', 'не', 'счастливая', 'семья', 'несчастлива', 'по', '-', 'своему', '.']
['T', 'u', 'tte', 'le', 'f', 'am', 'i', 'g', 'l', 'ie', 'f', 'e', 'l', 'ic', 'i', 's', 'on', 'o', 'si', 'm', 'il', 'i', 'le', 'une', 'a', 'lle', 'a', 'l', 't', 're', ';', 'o', 'g', 'ni', 'f', 'am', 'i', 'g', 'l', 'i', 'a', 'in', 'f', 'e', 'l', 'i', 'ce', '[UNK]', 'in', 'f', 'e', 'l', 'i', 'ce', 'a', 'mo', 'd', 'o', 's', 'u', 'o', '.']
